In [1]:
!nvidia-smi

Wed Sep 29 06:32:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:18:00.0 Off |                    0 |
| N/A   43C    P0    42W / 250W |   1181MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000000:AF:00.0 Off |                    0 |
| N/A   

In [2]:
from argparse import Namespace
import logging
opt = {
    "batch_size": 4,
    "num_workers": 4,
    "lr": 1e-4,
    "max_epochs": 10,
    "warmup_ratio": 0.2,
    "print_step": 100,
    "save_path": "model_weights",
} 
args = Namespace(**opt)

In [3]:
import json
path = "dacon/"
train_file = path + "train.json"
with open(train_file, "r") as f:
    TRAIN_DATA = json.load(f)

In [4]:
import json
path = "dacon/"
test_file = path + "test.json"
with open(test_file, "r") as f:
    TEST_DATA = json.load(f)

In [10]:
from kobart import get_kobart_tokenizer
kobart_tokenizer = get_kobart_tokenizer()
kobart_tokenizer.tokenize("안녕하세요. 한국어 BART 입니다.🤣:)l^o")

using cached model


['▁안녕하', '세요.', '▁한국어', '▁B', 'A', 'R', 'T', '▁입', '니다.', '🤣', ':)', 'l^o']

In [5]:
from transformers import BartModel, BartForConditionalGeneration
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer
kobart_tokenizer = get_kobart_tokenizer()
model = BartForConditionalGeneration.from_pretrained(get_pytorch_kobart_model())#BartModel.from_pretrained(get_pytorch_kobart_model())
inputs = kobart_tokenizer(['안녕하세요.'], return_tensors='pt')
model(inputs['input_ids'])

using cached model
using cached model


Seq2SeqLMOutput(loss=None, logits=tensor([[[-3.1152,  5.6895, -8.4704,  ..., -7.6067, -4.6915, -4.2773],
         [-6.7131,  4.5574, -7.6131,  ..., -6.8429, -5.2066, -6.7749]]],
       grad_fn=<AddBackward0>), past_key_values=((tensor([[[[-9.7980e-02, -6.6584e-01, -1.8089e+00,  ...,  9.6023e-01,
           -1.8818e-01, -1.3252e+00],
          [-6.2507e-01,  5.1009e-01, -7.4878e-01,  ...,  8.6230e-01,
            1.5722e-01, -6.0267e-01]],

         [[ 5.4597e-01, -2.3990e-01,  1.5901e+00,  ...,  4.3655e-01,
            7.9514e-01,  8.9880e-02],
          [-1.7327e-01, -6.3167e-01,  4.5152e-02,  ..., -1.4111e-01,
            1.8678e-01, -1.2081e-01]],

         [[ 1.4621e+00,  1.8980e+00, -7.6696e-01,  ...,  1.5695e+00,
            6.7921e-02, -3.9372e-01],
          [-4.1204e-02,  1.7132e+00, -1.1863e+00,  ..., -2.2272e-01,
            9.8310e-02,  8.1729e-01]],

         ...,

         [[ 4.8868e-01,  1.2633e+00, -1.1658e-01,  ..., -3.1989e-01,
            1.2202e+00, -7.9021e-02],
  

In [6]:
import pandas as pd
train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
uid = 1000
for data in TRAIN_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        train.loc[uid, 'uid'] = uid
        train.loc[uid, 'title'] = data['title']
        train.loc[uid, 'region'] = data['region']
        train.loc[uid, 'context'] = context[:-1]
        train.loc[uid, 'summary'] = data['label'][agenda]['summary']
        uid += 1

test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
uid = 2000
for data in TEST_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        test.loc[uid, 'uid'] = uid
        test.loc[uid, 'title'] = data['title']
        test.loc[uid, 'region'] = data['region']
        test.loc[uid, 'context'] = context[:-1]
        uid += 1

In [7]:
train['total'] = train.title + ' ' + train.region + ' ' + train.context
test['total'] = test.title + ' ' + test.region + ' ' + test.context


In [8]:
df_train = train.iloc[:-200]
df_val = train.iloc[-200:]

In [9]:
import os
import glob
import torch
import ast
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from torch.utils.data import Dataset, DataLoader, IterableDataset

class KoBARTSummaryDataset(Dataset):
    def __init__(self, file, tok, max_len=512, pad_index = 0, ignore_index=-100, train=True):
        super().__init__()
        self.tok = tok
        self.max_len = max_len
        self.docs = file
        self.len = len(self.docs)
        self.pad_index = pad_index
        self.ignore_index = ignore_index
        self.train = train

    def add_padding_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.pad_index] *(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]

        return inputs

    def add_ignored_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.ignore_index] *(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]

        return inputs
    
    def __getitem__(self, idx):
        instance = self.docs.iloc[idx]
        context = instance['context']
        if self.train:
            summary = instance['summary']
        input_ids = self.tok.encode(context)
        input_ids = self.add_padding_data(input_ids)

        if self.train:
            label_ids = self.tok.encode(instance['summary'])
            label_ids.append(self.tok.eos_token_id)
            dec_input_ids = [self.pad_index]
            dec_input_ids += label_ids[:-1]
            dec_input_ids = self.add_padding_data(dec_input_ids)
            label_ids = self.add_ignored_data(label_ids)


            return {'input_ids': np.array(input_ids, dtype=np.int_),
                    'decoder_input_ids': np.array(dec_input_ids, dtype=np.int_),
                    'labels': np.array(label_ids, dtype=np.int_)}
        else:
            return {'input_ids': np.array(input_ids, dtype=np.int_),}
    
    def __len__(self):
        return self.len

In [10]:
test_dataset = KoBARTSummaryDataset(file=test, tok=kobart_tokenizer, train=False)
test_dataloader = DataLoader(test_dataset, batch_size=1, num_workers=args.num_workers, shuffle=False)

In [11]:
import torch
import torch.nn as nn
class KobartSummaryModule(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.device = device
        self.model = BartForConditionalGeneration.from_pretrained(get_pytorch_kobart_model()).to(self.device)
        
    def forward(self, input, train=True):
        input_ids = torch.as_tensor(input["input_ids"]).to(self.device)
        if train:
            decoder_input_ids = torch.as_tensor(input["decoder_input_ids"]).to(self.device)
            labels = torch.as_tensor(input["labels"]).to(self.device)

            if len(input_ids.shape) == 1:
                input_ids, decoder_input_ids, labels = input_ids.unsqueeze(0), decoder_input_ids.unsqueeze(0), labels.unsqueeze(0)
            
            return_dict = self.model(input_ids=input_ids, decoder_input_ids=decoder_input_ids, labels=labels) 

            loss = return_dict["loss"]

            return loss, return_dict
        else:
            if len(input_ids.shape) == 1:
                input_ids = input_ids.unsqueeze(0)
                
            return_dict = self.model(input_ids=input_ids)

            return return_dict

    def generate(self, input, num_beams=4, max_length=50, early_stopping=True):
        input_ids = torch.as_tensor(input["input_ids"]).to(self.device)
        if len(input_ids.shape) == 1:
                input_ids = input_ids.unsqueeze(0)

        return self.model.generate(input_ids, num_beams=num_beams, max_length=max_length, early_stopping=early_stopping)

        

        


In [12]:
device = "cuda"

model = KobartSummaryModule(device=device)
model.load_state_dict(torch.load(args.save_path + "/model_1.pth"))
#return_dict = model(input, train=False)
#logits = return_dict["logits"]

using cached model


<All keys matched successfully>

In [23]:
#summary_ids = model.generate(input, num_beams=4, max_length=100, early_stopping=True)
#print([kobart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

In [20]:
def decode_summary(summary_ids):
    return [kobart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]


In [21]:
def summarization(i):
    context = test.iloc[i]["context"]
    input = test_dataset[i]
    summary_ids = model.generate(input, num_beams=4, max_length=100, early_stopping=True)
    summary = decode_summary(summary_ids)

    return{"context": context, "summary": summary}

In [22]:
summarization(25)

{'context': '의사일정 제7항, 음성군 맹동면ㆍ진천군 덕산면 택시공동사업구역 확대 지정 촉구 건의안을 상정합니다. 대표발의하신 김윤희 의원님께서는 나오셔서 본 안건에 대하여 제안설명해 주시기 바랍니다. 김윤희 의원입니다. 존경하는 동료 의원 여러분 그리고 이필용 군수님을 비롯한 관계공무원 여러분! 제가 대표발의한 음성군 맹동면과 진천군 덕산면 택시공동사업구역 확대 지정 촉구 건의안에 대하여 제안설명드리겠습니다. 충청북도의 충북혁신도시 택시공동사업구역 직권 지정에 대하여 음성군 맹동면과 진천군 덕산면 지역 전체에 대해서도 금년 상반기 중에 택시공동사업구역 지정이 절실하여 본 건의안을 제안하게 되었습니다. 그러면 유인물에 따라 본 건의문을 낭독하겠습니다. 「음성군 맹동면ㆍ진천군 덕산면 택시공동사업구역 확대 지정 촉구 건의문」 음성군의회 의원 일동은 지난 2018년 2월 2일 음성군 택시업계의 합리적 제안을 배제한 채 진천군 택시업계의 입장만 반영한 충청북도의 일방적인 충북혁신도시 택시공동사업구역 직권 지정에 대하여 매우 유감을 표명하는 바입니다. 충북혁신도시는 2014년 6월 아파트 입주가 시작했음에도 여전히 미비한 정주여건, 적은 입주민과 유동인구로 택시이용객이 적어 적자운행이 불가피했습니다. 그러나 음성군 택시업계는 적자운행에도 불구하고 2015년 3월부터 혁신도시 내 순번제 운행을 시작하여 주민 교통편의를 제공해 왔으며, 현재도 11대의 택시를 상주시켜 영업을 함으로써 택시 이용객들의 편의제공을 위하여 노력하고 있습니다. 금번 택시공동사업구역 직권 지정은 음성군과 진천군 택시요금이 동일한 체계에서 혁신도시만 지정되었기에 요금에는 전혀 변화가 없으며, 혁신도시 이외의 지역으로 운행할 경우에는 시계 외 할증이 부과되어 이로 인해 요금시비를 우려한 승차거부 문제와 택시요금과 관련한 주민불만이 해소되지 않을 것입니다. 또한, 최근 혁신도시 인구 및 상업시설 입주 증가와 더불어 혁신도시 경계 밖 맹동면, 덕산면 지역 개발에 따라 혁신도시 인접지역에서의 택시 